In [ ]:
# SH-I

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np

from encoder import encode, vectorize, classify

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
lectin = 'G3C' # which lectin
epochs = 25 # how many epochs
lr = 1.0 # how fast to learn
bs = 16 # batch size

In [ ]:
data = pd.read_csv('data/G3C.csv')

In [ ]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(920, 30)
        self.fc2 = nn.Linear(30, 3)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def predict(glycan):
    
    with torch.no_grad():
        output = net(encode(glycan).unsqueeze(0).to(device))
        prediction = classify(output)
        
    return prediction

In [ ]:
test_list = []

for i in range(len(data)):
    
    train_data = list(zip(data['IUPAC'], data['binding']))
    test_glycan, test_target = train_data.pop(i)
    trainset = [(encode(iupac), vectorize(binding)) for (iupac, binding) in train_data]
    trainloader = torch.utils.data.DataLoader(trainset, batch_size = bs, shuffle = True)
    print(i + 1, test_glycan, 'held out')
    
    net = Net()
    net = net.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(net.parameters(), lr = lr)
    
    for epoch in range(epochs):
        running_loss = 0.0
    
        for i, (inputs, labels) in enumerate(trainloader, 0):
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels.argmax(dim = 1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print('loss:', round(running_loss, 4))
        
    print('Finished Training')
    prediction = predict(test_glycan)
    test_list.append(prediction)
    print('Prediction:', prediction)
    print('Actual:', test_target, '\n')
    break

In [ ]:
data['prediction'] = test_list
data.to_csv('%s_predicted_epochs%s_lr%s_hn%s.csv' % (lectin, str(epochs), str(lr), str(hn)), index = False)